In [ ]:
import matplotlib.pyplot as plt
import scipy.stats as st
import yfinance as yf
import seaborn as sns
import pandas as pd
import numpy as np
import datetime

In [ ]:
# currency in which the simulation is run
analysis_currency = "EUR"

# start and end date of the simulation
# if start_date is before the first available date of the securities, the first available date will be used
start_date = "2019-08-01"
end_date = "2023-11-01"
# end_date = datetime.datetime.today().strftime("%Y-%m-%d")

# tickers and the corresponding currencies of securities to download from yahoo finance
tickers_and_currencies = {
    "VWCE.DE": "EUR",
    "ISAC.L": "USD",
    "VAGP.L": "GBP",
    "AGGH.MI": "EUR",
    "4GLD.DE": "EUR",
    "IGLN.L": "USD",
}

# weights of the securities in the portfolio
# the transactions will made in order to keep the weights
weights = [0.3, 0.2, 0.1, 0.2, 0.1, 0.1]

# amount of expense (transaction fee) for each transaction
expense = 1

# approximate amount of money to invest in each transaction
amount = 1000

# which column to use for open, high, low, close prices
ohlc = "close"

# number of iterations for the monte carlo simulation
iterations = 1000

# every n months a transaction will be made
n_months = 1

In [ ]:
# take securities names from tickers_and_currencies dictionary
securities = [security_name.split(".")[0] for security_name in tickers_and_currencies]

# take tickers and currencies from tickers_and_currencies dictionary
tickers = [*tickers_and_currencies.keys()]
currencies_securities = [*tickers_and_currencies.values()]

# take all currencies
currencies = currencies_securities + [analysis_currency]

# take distinct currencies
distinct_currencies = list(set(currencies))

# convert ohlc variable to valid format
ohlc = ohlc[0].upper() + ohlc[1:].lower()

In [ ]:
# download securities data from yahoo finance
yahoo_securities_data = yf.download(tickers, period="max")[ohlc]

# make yahoo_securities_data dataframe
securities_data = pd.DataFrame(yahoo_securities_data)

# set columns order to a specified one in order to correctly set columns names later
securities_data = securities_data[tickers]

# set index name to 'Date' and set columns names to securities names
securities_data.index.name = "Date"
securities_data.columns = securities

In [ ]:
# create list of currency pairs to download exchange rates for
distinct_currency_pairs = [
    currency + analysis_currency for currency in distinct_currencies
]

# remove currency pair which is the same as analysis currency
distinct_currency_pairs.remove(analysis_currency * 2)

# if currency pairs are specified then download exchange rates for them else set exchange rates to None
distinct_currency_pairs_format = [
    currency + "=X" for currency in distinct_currency_pairs
]

# download currency data from yahoo finance
yahoo_currency_data = yf.download(distinct_currency_pairs_format, period="max")[ohlc]

# make yahoo_currency_data dataframe
exchange_rates = pd.DataFrame(yahoo_currency_data)

# set columns order to a specified one in order to correctly set columns names later
exchange_rates = exchange_rates[distinct_currency_pairs_format]
exchange_rates.columns = distinct_currency_pairs

# if there is analysis currency in distinct currencies then add column with exchange rates equal to 1.0
if distinct_currencies.index(analysis_currency) != -1:
    analysis_currency_exchange_rates = pd.Series(
        [1.0 for _ in range(len(securities_data.index))],
        index=securities_data.index,
        name=analysis_currency * 2,
    )
    exchange_rates = pd.concat(
        [exchange_rates, analysis_currency_exchange_rates], axis=1
    )

In [ ]:
# make a copy of securities_data dataframe to use it for simulation
# copy to reuse securities_data dataframe if needed without downloading data again
simulation_data = securities_data.copy()

# convert simulation_data dataframe to the currency of analysis
for ticker, currency in tickers_and_currencies.items():
    security_name = ticker.split(".")[0]
    currency_pair = currency + analysis_currency
    simulation_data[security_name] = (
        simulation_data[security_name] * exchange_rates[currency_pair]
    )

In [ ]:
# drop rows with NaN values as it is not possible to simulate portfolio with them
simulation_data = simulation_data.dropna()

In [ ]:
securities_names = [name[:4] for name in tickers]

securities_unit_value = [name + "_UNIT_VALUE" for name in securities_names]
securities_count = [name + "_COUNT" for name in securities_names]
securities_value = [name + "_VALUE" for name in securities_names]
securities_expense = [name + "_EXPENSE" for name in securities_names]
securities_buy_moment_expense = [
    name + "_BUY_MOMENT_EXPENSE" for name in securities_names
]
securities_buy_moment_count = [name + "_BUY_MOMENT_COUNT" for name in securities_names]
securities_profit = [name + "_PROFIT" for name in securities_names]

# portfolio columns names
portfolio_count = "PORTFOLIO_COUNT"
portfolio_value = "PORTFOLIO_VALUE"
portfolio_expense = "PORTFOLIO_EXPENSE"
portfolio_profit = "PORTFOLIO_PROFIT"
portfolio_profit_percentage = "PORTFOLIO_PROFIT_PERCENTAGE"
portfolio_drawdown = "PORTFOLIO_DRAWDOWN"
portfolio_cagr = "PORTFOLIO_CAGR"

In [ ]:
# change columns names to the ones with suffix
simulation_data.columns = securities_unit_value

# add columns with suffix and set values to 0
simulation_data[securities_count] = 0

In [ ]:
# function that generates buy transaction dates every n months
def generate_buy_transaction_dates(simulation_data):
    buy_transaction_dates = []
    dates_groups = simulation_data.groupby(
        [simulation_data.index.year, simulation_data.index.month]
    )

    for i, (_, group) in enumerate(dates_groups):
        if i % n_months == 0:
            buy_transaction_dates.append(group.sample(1).index[0])

    return buy_transaction_dates

In [ ]:
# dictionary with statistics of profit for each iteration
results_profit_describe = {}

# dictionary with statistics of profit percentage for each iteration
results_profit_percentage_describe = {}

# dictionary with statistics of drawdown for each iteration
results_drawdown_describe = {}

# dictionary with statistics of CAGR for each iteration
results_cagr_describe = {}

# dictionary with profits pandas series for each iteration
portfolio_profits = {}

In [ ]:
for iteration in range(iterations):
    # make a copy of simulation_data dataframe to use it for simulation
    simulation_data_copy = simulation_data.copy()

    # generate buy transaction dates
    buy_transaction_dates = generate_buy_transaction_dates(simulation_data_copy)

    # take only rows with dates greater than the first buy transaction date
    simulation_data_copy = simulation_data_copy.loc[
        simulation_data_copy.index >= buy_transaction_dates[0]
    ].copy()

    # loop through buy transaction dates
    for date in buy_transaction_dates:
        # cumulative sum of securities counts to get current counts
        counts_cumsum = simulation_data_copy[securities_count].cumsum()

        # cumulative sum of securities counts to get counts for a given date
        counts_cumsum = simulation_data_copy[securities_count].cumsum()

        # get securities counts for a given date
        counts_for_date = counts_cumsum.loc[date]

        # get securities unit values for a given date
        unit_values_for_date = simulation_data_copy.loc[date, securities_unit_value]

        # get current values of securities by multiplying currect counts and unit values
        current_values = counts_for_date.values * unit_values_for_date.values

        # get current values sum
        current_values_sum = current_values.sum()

        # get weights deviations by subtracting current share of each security from the target share
        # if current_values_sum is not 0 else set weights deviations to list of 0s
        if current_values_sum:
            weights_deviations = [
                weight - current_value / current_values_sum
                for weight, current_value in zip(weights, current_values)
            ]
        else:
            weights_deviations = [0] * len(weights)

        # get weights model plus deviations by adding weights and their deviations
        # positive weights deviations mean that the security is underweighted and vice versa
        weights_model_plus_deviations = [
            weight + deviation for weight, deviation in zip(weights, weights_deviations)
        ]

        # get security amounts by multiplying weights model plus deviations by the amount
        security_amounts = [amount * weight for weight in weights_model_plus_deviations]

        # get buy counts by dividing security amounts by securities unit values
        buy_counts = (
            security_amounts / simulation_data_copy.loc[date, securities_unit_value]
        )

        # round down buy_counts to the integer and add 0.1 to round up the values with 0.9 decimal part
        buy_counts = buy_counts.apply(lambda x: int(x + 0.1))

        # assign new transaction buy counts and expense to a given date in simulation_data_copy dataframe
        simulation_data_copy.loc[date, securities_count] = buy_counts.values
        simulation_data_copy.loc[date, securities_expense] = [expense] * len(
            securities_names
        )

    # assign securities counts to expenses and buy moment counts
    simulation_data_copy[securities_expense] = simulation_data_copy[securities_count]
    simulation_data_copy[securities_buy_moment_count] = simulation_data_copy[
        securities_count
    ]

    # assign securities count cummulatives sums to counts and values
    simulation_data_copy[securities_count] = simulation_data_copy[
        securities_count
    ].cumsum()
    simulation_data_copy[securities_value] = simulation_data_copy[securities_count]

    # multiply securities values by securities unit values to get securities values based on transaction counts
    for security_value, security_unit_value in zip(
        securities_value, securities_unit_value
    ):
        simulation_data_copy[security_value] = (
            simulation_data_copy[security_value]
            * simulation_data_copy[security_unit_value]
        )

    # multiply securities expenses by securities unit values to get securities expenses based on transaction counts and add expense variable
    for security_expense, security_unit_value in zip(
        securities_expense, securities_unit_value
    ):
        simulation_data_copy[security_expense] = (
            simulation_data_copy[security_expense]
            * simulation_data_copy[security_unit_value]
        )
        simulation_data_copy.loc[
            simulation_data_copy[security_expense] > 0, security_expense
        ] += expense

    # assign securities expenses to buy moment expenses and calculate cumulative sum of securities expenses
    simulation_data_copy[securities_buy_moment_expense] = simulation_data_copy[
        securities_expense
    ]
    simulation_data_copy[securities_expense] = simulation_data_copy[
        securities_expense
    ].cumsum()

    # calculate securities profits
    for security_profit, security_value, security_expense in zip(
        securities_profit, securities_value, securities_expense
    ):
        simulation_data_copy[security_profit] = (
            simulation_data_copy[security_value]
            - simulation_data_copy[security_expense]
        )

    # calculate portfolio values, expenses, profits and profit percentages
    simulation_data_copy[portfolio_value] = simulation_data_copy[securities_value].sum(
        axis=1
    )
    simulation_data_copy[portfolio_expense] = simulation_data_copy[
        securities_expense
    ].sum(axis=1)
    simulation_data_copy[portfolio_profit] = (
        simulation_data_copy[portfolio_value] - simulation_data_copy[portfolio_expense]
    )
    simulation_data_copy[portfolio_profit_percentage] = (
        simulation_data_copy[portfolio_profit] / simulation_data_copy[portfolio_expense]
    ) * 100

    # calculate portfolio drawdowns
    for date in simulation_data_copy.index:
        # calculate max portfolio value for the current date
        max_value = simulation_data_copy.loc[:date, portfolio_value].max()

        # take current portfolio value
        current_value = simulation_data_copy.loc[date, portfolio_value]

        # if max_value is 0 then set drawdown to 0 else calculate drawdown
        if max_value == 0:
            simulation_data_copy.loc[date, portfolio_drawdown] = 0
            continue

        # calculate drawdown by subtracting current value from max value and dividing by max value
        simulation_data_copy.loc[date, portfolio_drawdown] = (
            current_value - max_value
        ) / max_value

    # calculate portfolio cagr
    for date in buy_transaction_dates:
        # calculate datediff between the last date and the current date
        datediff = simulation_data_copy.index[-1] - date

        # if datediff is less than 365 days then break the loop
        if datediff.days < 365:
            break

        # calculate future value by multiplying last securities unit values by securities buy moment counts for the current date and summing them
        future_value = (
            simulation_data_copy.iloc[-1].loc[securities_unit_value].values
            * simulation_data_copy.loc[date, securities_buy_moment_count].values
        )
        future_value = sum(future_value)

        # calculate cagr by dividing future value by securities buy moment expenses sum and raising to the power of 365 / datediff.days
        simulation_data_copy.loc[date, portfolio_cagr] = (
            future_value
            / simulation_data_copy.loc[date, securities_buy_moment_expense].sum()
        ) ** (365 / datediff.days) - 1

    # -------------------- add iteration results to results dictionaries -------------------- #

    # add portfolio profit describe to a results dictionary
    results_profit_describe[iteration] = simulation_data_copy[
        portfolio_profit
    ].describe()

    # add portfolio profit percentage describe to a results dictionary
    results_profit_percentage_describe[iteration] = simulation_data_copy[
        portfolio_profit_percentage
    ].describe()

    # add portfolio max drawdown describe to a results dictionary
    results_drawdown_describe[iteration] = simulation_data_copy[
        portfolio_drawdown
    ].describe()

    # add portfolio cagr describe to a results dictionary
    results_cagr_describe[iteration] = simulation_data_copy[portfolio_cagr].describe()

    # add portfolio profit to a results dictionary
    portfolio_profits[iteration] = simulation_data_copy[portfolio_profit]

    print(f"Iteration {iteration + 1} out of {iterations} is done")

In [ ]:
# make dataframes from results dictionaries

df_profit_describe = pd.DataFrame.from_dict(results_profit_describe, orient="index")
df_profit_percentage_describe = pd.DataFrame.from_dict(
    results_profit_percentage_describe, orient="index"
)
df_drawdown_describe = pd.DataFrame.from_dict(results_drawdown_describe, orient="index")
df_cagr_describe = pd.DataFrame.from_dict(results_cagr_describe, orient="index")

# drop columns with NaN values as it is not possible to simulate portfolio with them correctly
df_portfolio_profits = (
    pd.DataFrame.from_dict(portfolio_profits, orient="index")
    .dropna(axis=1)
    .sort_index()
)

In [ ]:
# function that plots confidence interval with 95% confidence level


def ci_plot(data, title, x_label, precision):
    interval = st.t.interval(
        confidence=0.95, df=len(data) - 1, loc=np.mean(data), scale=np.std(data)
    )

    fig, ax = plt.subplots(figsize=(12, 8))
    ax = sns.histplot(x=data, color="darkgreen", kde=True, stat="count")
    ax.lines[0].set_color("black")
    ax.axvline(interval[0], color="r")
    ax.axvline(interval[1], color="r")
    ax.set_title(
        f"{title}\nconfidence interval (95%): [{interval[0]:.{precision}f}, {interval[1]:.{precision}f}]"
    )
    ax.set_xlabel(x_label)
    plt.show()

In [ ]:
# calculate mean profit for each date over all iterations
df_portfolio_profits_mean = df_portfolio_profits.mean(axis=0)
df_portfolio_profits_ci = pd.DataFrame(
    index=df_portfolio_profits_mean.index, columns=["lower", "upper"]
)

# calculate confidence interval for each date over all iterations
for date in df_portfolio_profits.columns:
    lower, upper = st.t.interval(
        confidence=0.95,
        df=len(df_portfolio_profits) - 1,
        loc=np.mean(df_portfolio_profits[date]),
        scale=np.std(df_portfolio_profits[date]),
    )
    (
        df_portfolio_profits_ci.loc[date, "lower"],
        df_portfolio_profits_ci.loc[date, "upper"],
    ) = (lower, upper)

# convert confidence interval values to numpy arrays
ci_lower = df_portfolio_profits_ci["lower"].to_numpy(dtype=np.float64)
ci_upper = df_portfolio_profits_ci["upper"].to_numpy(dtype=np.float64)

# plot mean profit for each date over all iterations
plt.figure(figsize=(12, 8))
plt.xlabel("Date")
plt.ylabel(f"Profit value [{analysis_currency}]")
plt.grid(True)
plt.title(f"Mean profits for each date over {iterations} iterations")
plt.plot(
    df_portfolio_profits_mean.index,
    df_portfolio_profits_mean.values,
    label="Profit",
    color="darkblue",
)
plt.axhline(y=0, color="black", linestyle="--")

# add error bands made with confidence interval (95%) values
plt.fill_between(df_portfolio_profits_mean.index, ci_lower, ci_upper, alpha=0.2)

# plot trend line
# x_values = pd.to_numeric(df_portfolio_profits_mean.index)
# poly_fit = np.polyfit(x_values, df_portfolio_profits_mean.values, deg=1)
# trend_line = np.poly1d(poly_fit)
# plt.plot(df_portfolio_profits_mean.index, trend_line(x_values), label="Trend Line", color="red", linestyle="--")

plt.show()

In [ ]:
# plot mean profit for each iteration with confidence interval (95%)
ci_plot(
    df_profit_describe["mean"],
    title=f"Mean profit for {iterations} iterations",
    x_label="Profit mean",
    precision=2,
)

In [ ]:
# boxplot with the maximum profit for each iteration

plt.figure(figsize=(8, 8))
boxplot = df_profit_describe.boxplot(
    column="max",
    patch_artist=True,
    medianprops={"color": "black"},
    whiskerprops=dict(color="blue", linestyle="--"),
    flierprops=dict(marker="o", markerfacecolor="red", markersize=6),
)
boxplot.set_title(f"Maximum profit for {iterations} iterations")
plt.ylabel(f"Profit value [{analysis_currency}]")
plt.show()

In [ ]:
# boxplot with the maximum percentage profit for each iteration

plt.figure(figsize=(8, 8))
boxplot = df_profit_percentage_describe.boxplot(
    column="max",
    patch_artist=True,
    medianprops={"color": "black"},
    whiskerprops=dict(color="blue", linestyle="--"),
    flierprops=dict(marker="o", markerfacecolor="red", markersize=6),
)
boxplot.set_title(f"Maximum percentage profit for {iterations} iterations")
plt.ylabel(f"Profit value %")
plt.show()

In [ ]:
# boxplot with the minimum profit for each iteration

plt.figure(figsize=(8, 8))
boxplot = df_profit_describe.boxplot(
    column="min",
    patch_artist=True,
    medianprops={"color": "black"},
    whiskerprops=dict(color="blue", linestyle="--"),
    flierprops=dict(marker="o", markerfacecolor="red", markersize=6),
)
boxplot.set_title(f"Minimum profit for {iterations} iterations")
plt.ylabel(f"Profit value [{analysis_currency}]")
plt.show()

In [ ]:
# boxplot with the minimum percentage profit for each iteration

plt.figure(figsize=(8, 8))
boxplot = df_profit_percentage_describe.boxplot(
    column="min",
    patch_artist=True,
    medianprops={"color": "black"},
    whiskerprops=dict(color="blue", linestyle="--"),
    flierprops=dict(marker="o", markerfacecolor="red", markersize=6),
)
boxplot.set_title(f"Minimum percentage profit for {iterations} iterations")
plt.ylabel(f"Profit value %")
plt.show()

In [ ]:
# plot with the mean drawdown for each iteration

ci_plot(
    df_drawdown_describe["mean"],
    title=f"Mean drawdown for {iterations} iterations",
    x_label="Drawdown mean",
    precision=4,
)

In [ ]:
# boxplot with the lowest drawdowns for each iteration

plt.figure(figsize=(8, 8))
boxplot = df_drawdown_describe.boxplot(
    column="min",
    patch_artist=True,
    medianprops={"color": "black"},
    whiskerprops=dict(color="blue", linestyle="--"),
    flierprops=dict(marker="o", markerfacecolor="red", markersize=6),
)
boxplot.set_title(f"Minimum drawdown for {iterations} iterations")
plt.ylabel(f"Drawdown value")
plt.show()

In [ ]:
# plot with the mean CAGR for each iteration
# CAGR is calculated for each transaction that was made at least 365 days from the last available date

ci_plot(
    df_cagr_describe["mean"],
    title=f"Mean CAGR for {iterations} iterations",
    x_label="CAGR mean",
    precision=4,
)

In [ ]:
# boxplot with the best transactions CAGR for each iteration

plt.figure(figsize=(8, 8))
boxplot = df_cagr_describe.boxplot(
    column="max",
    patch_artist=True,
    medianprops={"color": "black"},
    whiskerprops=dict(color="blue", linestyle="--"),
    flierprops=dict(marker="o", markerfacecolor="red", markersize=6),
)
boxplot.set_title(f"Maximum CAGR for {iterations} iterations")
plt.ylabel(f"CAGR value")
plt.show()

In [ ]:
# boxplot with the worst transactions CAGR for each iteration

plt.figure(figsize=(8, 8))
boxplot = df_cagr_describe.boxplot(
    column="min",
    patch_artist=True,
    medianprops={"color": "black"},
    whiskerprops=dict(color="blue", linestyle="--"),
    flierprops=dict(marker="o", markerfacecolor="red", markersize=6),
)
boxplot.set_title(f"Minimum CAGR for {iterations} iterations")
plt.ylabel(f"CAGR value")
plt.show()